# Solving Single Decisions

## The "Party Problem" example, Value of Information

__ A continuation of partyproblem_xdsl.ipynb__

JMA 17 Sept 2025

In [1]:
# Imports from the python standard library
import math, re, os, sys
from pathlib import Path
import itertools            # to flatten lists

# Import array and dataframe packages
import numpy as np
# import numpy.linalg as la
import pandas as pd

import networkx as nx

# for extract_net
# from ID_operations import * 
from potential_operations import *
import BN

## Bayes networks object

In [2]:
NETWORK_FILE = 'PartyProblem_asym.xdsl'
# BN structure is contained under the node branch
parsed = BN.extract_net(NETWORK_FILE)
nodes, extensions = parsed
# tags tell the node type. 
# [( k.get('id'), k.tag) for k in nodes]
# CPT contents are stored in row major order (first row, second row, ...)
# Parents are the first matrix dimension -- matrix is Row Markov
pp_net = BN.reap(parsed)
pp_net.pr_nodes()

found nodes
Edges:  [('Weather', 'Preferences'), ('Party_location', 'Preferences'), ('Weather', 'Detector'), ('Adjustor', 'Detector')] 

>>> Party_location <<<
Party_location: decision
	states: ['Outdoors', 'Porch', 'Indoors']
	named tensor: [('Party_location', 'm')], [3]
       [1, 1, 1]

>>> Weather <<<
Weather: cpt
	states: ['Sunshine', 'Rain']
	named tensor: [('Weather', 'm')], [2]
       [0.4000, 0.6000]

>>> Preferences <<<
Preferences: utility
	states: ['utility']
	named tensor: [('Weather', 'c'), ('Party_location', 'c'), ('Preferences', 'm')], [2, 3, 1]
       [[[100.],
         [ 90.],
         [ 40.]],

        [[  0.],
         [ 20.],
         [ 50.]]]

>>> Adjustor <<<
Adjustor: cpt
	states: ['absent', 'present']
	named tensor: [('Adjustor', 'm')], [2]
       [0.1000, 0.9000]

>>> Detector <<<
Detector: cpt
	states: ['SaySun', 'SayRain']
	named tensor: [('Weather', 'c'), ('Adjustor', 'c'), ('Detector', 'm')], [2, 2, 2]
       [[[0.5100, 0.4900],
         [0.8700, 0.1300]],

reap: Party_location, decision
reap: Weather, cpt
reap: Preferences, utility
reap: Utility, maux
Unsupported node type: maux
reap: Adjustor, cpt
reap: Detector, cpt


## Add Weather as a conditioning variable

Modify the decision Potential to include a dimension for the Weather variable

In [3]:
decn_w_observation = condition_decision(
    pp_net.get_potential('Party_location'), 
    pp_net.get_potential('Weather'))
decn_w_observation

OrderedDict({'Weather': 'c', 'Party_location': 'm'})
	tensor([[1, 1, 1],
        [1, 1, 1]])

### Solution order of node operations

- Add observation dimension to the Party_location decision
- ? marginalize Weather into value
- Compute EV for each option, conditioned on weather state
- Maximize over options (conditioned on weather)
- Expect out weather to get EV